## Goal: Determine the performance of the current inbound forecast for 2022

## Functions

### Load xls parser

In [0]:
#  !pip install holidays 

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

### Functions to eval performance

In [0]:
import pandas as pd
from typing import Literal, Optional
import plotly.graph_objects as go
import holidays
from datetime import datetime


In [0]:
def calculate_errors(df_eval:pd.DataFrame, actuals_col:str,fc_col:str, error_to_calculate:Literal["Bias", "AbsError", "SqError","RelAbsError"], error_colname_suffix:str, outside_range_threshold:int=0.1)->pd.DataFrame:
    """
    Calculates error metrics based on the provided data. Bear in mind that this function appends the columns to the existing dataframe.

    Args:
        df_eval (pd.DataFrame): The DataFrame containing the evaluation data.
        actuals_col (str): The name of the column in `df_eval` that represents the actual values.
        fc_col (str): The name of the column in `df_eval` that represents the forecasted values.
        error_to_calculate (Literal["Bias", "AbsError", "SqError", "RelAbsError"]): The type of error to calculate. 
            Possible values:
                - "Bias": Calculate the bias between forecasted and actual values.
                - "AbsError": Calculate the absolute error between forecasted and actual values.
                - "SqError": Calculate the squared error between forecasted and actual values.
                - "RelAbsError": Calculate the relative absolute error between forecasted and actual values.
        error_colname_suffix (str): The suffix to append to the error column name in `df_eval`.
        outside_range_threshold (int): Threshold for the permissible error range of the relative error. Default 0.1 

    Returns:
        pd.DataFrame: The DataFrame `df_eval` with the calculated error column added.

    Raises:
        ValueError: If the `error_to_calculate` argument is not one of the valid error types.

    Examples:
        >>> data = pd.DataFrame({'actuals': [1, 2, 3], 'forecast': [1.2, 2.5, 2.8]})
        >>> calculate_errors(data, 'actuals', 'forecast', 'AbsError', 'error')
           actuals  forecast  AbsError_error
        0        1       1.2             0.2
        1        2       2.5             0.5
        2        3       2.8             0.2
    """
    error_colname = error_to_calculate+'_'+ error_colname_suffix
    # Calculate the bias
    df_eval[error_colname] = df_eval[fc_col]-df_eval[actuals_col]
    if (error_to_calculate == 'AbsError')|(error_to_calculate == 'RelAbsError'):
      df_eval[error_colname] = df_eval[error_colname].abs()
      if error_to_calculate == 'RelAbsError':
        df_eval[error_colname] = df_eval[error_colname].div(df_eval[actuals_col])
        df_eval['Outside_range_'+ error_colname_suffix] = df_eval[error_colname]>outside_range_threshold
    if error_to_calculate == 'SqError':
      df_eval[error_colname] = df_eval[error_colname].pow(2)
    
    return df_eval


In [0]:
def plot_inbound_fc(inbound_df:pd.DataFrame, actuals_col:str, fc_col:str, show_out_of_range:bool, date_col:str, outside_range_col:Optional[str]=None):
    """
    Plots the forecasted and actual values from the provided DataFrame `inbound_df`,
    along with the outside range data points.

    Args:
        inbound_df (pd.DataFrame): The DataFrame containing the inbound data.
        actuals_col (str): The name of the column in `inbound_df` that represents the actual values.
        fc_col (str): The name of the column in `inbound_df` that represents the forecasted values.
        show_out_of_range (bool): Plots a vertical line on the days where the forecast was out of range
        outside_range_col (str): The name of the column in `inbound_df` that represents whether a data point is outside the range (defined with a bool flag).
        date_col (str): The name of the column in `inbound_df` that represents the dates.

    Returns:
        None

    Examples:
        >>> data = pd.DataFrame({'Dates': ['2023-06-01', '2023-06-02', '2023-06-03'],
                                'fc_col': [1.2, 2.5, 2.8],
                                'actuals_col': [1, 2, 3],
                                'outside_range_col': [False, False, True]})
        >>> plot_inbound_fc(data, 'actuals_col', 'fc_col',  True, 'Dates', 'outside_range_col')
        (Plot of forecasted and actual values with outside range data points displayed)
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=inbound_df.Dates, y=inbound_df[fc_col], name=fc_col,line=dict(color='#0000FF'),showlegend=True))
    fig.add_trace(go.Scatter(x=inbound_df.Dates, y=inbound_df[actuals_col], name=actuals_col,line=dict(color='#00FF00'),showlegend=True))
    if show_out_of_range:
      ## Add outside of range traces (this implementation was preferred over the vline method to have the legend included)
      outside_range_dates = inbound_df[inbound_df[outside_range_col]==True][date_col].reset_index(drop=True)
      boundaries_category = [inbound_df[actuals_col].min(),inbound_df[actuals_col].max()]
      for i in range(len(outside_range_dates)):
        showlegend = False
        if i ==0:
          showlegend = True
        fig.add_trace(go.Scatter(x=[outside_range_dates[i],outside_range_dates[i]], y= boundaries_category, mode='lines', line=dict(color='#FF0000', width=0.5, dash='dash'), name=outside_range_col,showlegend=showlegend))
    fig.show()

In [0]:
def plot_inbound_error(inbound_df:pd.DataFrame, error_col:str,  show_out_of_range:bool, date_col:str, outside_range_col:Optional[str]=None):
    """
    Plots the forecasting error from the provided DataFrame `inbound_df`,
    along with the outside range data points.

    Args:
        inbound_df (pd.DataFrame): The DataFrame containing the inbound data.
        error_col (str): The name of the column in `inbound_df` that represents the error values.
        show_out_of_range (bool): Plots a vertical line on the days where the forecast was out of range
        outside_range_col (str): The name of the column in `inbound_df` that represents whether a data point is outside the range (defined with a bool flag).
        date_col (str): The name of the column in `inbound_df` that represents the dates.

    Returns:
        None

    Examples:
        >>> data = pd.DataFrame({'Dates': ['2023-06-01', '2023-06-02', '2023-06-03'],
                                'error_col': [1.2, 2.5, 2.8], 
                                'outside_range_col': [False, False, True]})
        >>> plot_inbound_fc(data, 'error_col', True, 'Dates', 'outside_range_col')
        (Plot of error_col values with outside range data points displayed)
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=inbound_df.Dates, y=inbound_df[error_col], name=error_col,line=dict(color='#0000FF'),showlegend=True))
    if show_out_of_range:
      ## Add outside of range traces (this implementation was preferred over the vline method to have the legend included)
      outside_range_dates = inbound_df[inbound_df[outside_range_col]==True][date_col].reset_index(drop=True)
      boundaries_category = [inbound_df[error_col].min(),inbound_df[error_col].max()]
      for i in range(len(outside_range_dates)):
        showlegend = False
        if i ==0:
          showlegend = True
        fig.add_trace(go.Scatter(x=[outside_range_dates[i],outside_range_dates[i]], y= boundaries_category, mode='lines', line=dict(color='#FF0000', width=0.5, dash='dash'), name=outside_range_col,showlegend=showlegend))
    fig.show()

In [0]:
def describe_inbound_fc(inbound_fc:pd.DataFrame, cols_to_keep:list)->pd.DataFrame:
    """
    Generates descriptive statistics for the selected columns of the provided DataFrame `inbound_fc`.

    Args:
        inbound_fc (pd.DataFrame): The DataFrame containing the inbound forecast data.
        cols_to_keep (list): A list of column names to include in the descriptive statistics.

    Returns:
        pd.DataFrame: The DataFrame with descriptive statistics for the selected columns.

    Examples:
        >>> data = pd.DataFrame({'col1': [1, 2, 3], 'col2': [4, 5, 6], 'col3': ['A', 'B', 'C']})
        >>> describe_inbound_fc(data, ['col1', 'col2'])
          Metrics  col1  col2
        0   count   3.0   3.0
        1    mean   2.0   5.0
        2     std   1.0   1.0
        3     min   1.0   4.0
        4     25%   1.5   4.5
        5     50%   2.0   5.0
        6     75%   2.5   5.5
        7     max   3.0   6.0
    """
    described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()
    return described_df

## Parameters

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
actual_cols = ['Dry Actuals', 'Fresh','Frozen', 'Ultrafresh']
fc_cols = ['Dry Fc', 'Fresh Fc', 'Frozen Fc', 'Ultrafresh Fc']
error_suffixes = ['Dry', 'Fresh','Frozen', 'Ultrafresh']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Evaluate performance

### Load the inbound dataset

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)

### Calculate errors

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= inbound_2022_df, actuals_col=actual_cols[i], fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])

In [0]:
inbound_2022_df

,Dates,Dry Fc,Dry Actuals,Fresh Fc,Fresh,Frozen Fc,Frozen,Ultrafresh Fc,Ultrafresh,Bias_Dry,Bias_Fresh,Bias_Frozen,Bias_Ultrafresh,AbsError_Dry,AbsError_Fresh,AbsError_Frozen,AbsError_Ultrafresh,SqError_Dry,SqError_Fresh,SqError_Frozen,SqError_Ultrafresh,RelAbsError_Dry,Outside_range_Dry,RelAbsError_Fresh,Outside_range_Fresh,RelAbsError_Frozen,Outside_range_Frozen,RelAbsError_Ultrafresh,Outside_range_Ultrafresh
0,2022-01-03,85794.442754,84224.0,60969.286307,41000.0,49273.053738,5581.0,6538.217201,6538.217201,1570.442754,19969.286307,43692.053738,0.000000,1570.442754,19969.286307,43692.053738,0.000000,2.466290e+06,3.987724e+08,1.908996e+09,0.000000e+00,0.018646,False,0.487056,True,7.828714,True,0.000000,False
1,2022-01-04,65323.939111,49564.0,68517.000793,64233.0,6447.045743,6110.0,7288.014352,9673.000000,15759.939111,4284.000793,337.045743,-2384.985648,15759.939111,4284.000793,337.045743,2384.985648,2.483757e+08,1.835266e+07,1.135998e+05,5.688157e+06,0.317971,True,0.066695,False,0.055163,False,0.246561,True
2,2022-01-05,59060.496963,45890.0,59891.490569,61162.0,14752.884950,6061.0,8645.127518,9394.000000,13170.496963,-1270.509431,8691.884950,-748.872482,13170.496963,1270.509431,8691.884950,748.872482,1.734620e+08,1.614194e+06,7.554886e+07,5.608100e+05,0.287001,True,0.020773,False,1.434068,True,0.079718,False
3,2022-01-06,54098.434315,26976.0,54044.997867,55700.0,11884.161805,5042.0,8733.406013,7299.000000,27122.434315,-1655.002133,6842.161805,1434.406013,27122.434315,1655.002133,6842.161805,1434.406013,7.356264e+08,2.739032e+06,4.681518e+07,2.057521e+06,1.005428,True,0.029713,False,1.357033,True,0.196521,True
4,2022-01-07,48633.525710,37044.0,48691.064872,49534.0,7322.811052,5979.0,7745.598367,8240.000000,11589.525710,-842.935128,1343.811052,-494.401633,11589.525710,842.935128,1343.811052,494.401633,1.343171e+08,7.105396e+05,1.805828e+06,2.444330e+05,0.312858,True,0.017017,False,0.224755,True,0.060000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,2022-12-27,71459.270000,63764.0,67049.998422,68912.0,6642.104962,5615.0,9028.927578,11033.000000,7695.270000,-1862.001578,1027.104962,-2004.072422,7695.270000,1862.001578,1027.104962,2004.072422,5.921718e+07,3.467050e+06,1.054945e+06,4.016306e+06,0.120684,True,0.027020,False,0.182922,True,0.181643,True
294,2022-12-28,71916.320000,47680.0,62406.358606,57239.0,5883.382433,5253.0,9509.601394,10525.000000,24236.320000,5167.358606,630.382433,-1015.398606,24236.320000,5167.358606,630.382433,1015.398606,5.873992e+08,2.670159e+07,3.973820e+05,1.031034e+06,0.508312,True,0.090277,False,0.120004,True,0.096475,False
295,2022-12-29,60346.346000,39552.0,76619.568000,63748.0,5951.031775,6037.0,9399.318000,10458.000000,20794.346000,12871.568000,-85.968225,-1058.682000,20794.346000,12871.568000,85.968225,1058.682000,4.324048e+08,1.656773e+08,7.390536e+03,1.120808e+06,0.525747,True,0.201913,True,0.014240,False,0.101232,True
296,2022-12-30,52203.888000,61119.0,65006.548000,42234.0,5759.074282,6692.0,9096.132000,7824.000000,-8915.112000,22772.548000,-932.925718,1272.132000,8915.112000,22772.548000,932.925718,1272.132000,7.947922e+07,5.185889e+08,8.703504e+05,1.618320e+06,0.145865,True,0.539199,True,0.139409,True,0.162594,True


In [0]:
inbound_2022_df.columns.values

Out[25]: array(['Dates', 'Dry Fc', 'Dry Actuals', 'Fresh Fc', 'Fresh', 'Frozen Fc',
       'Frozen', 'Ultrafresh Fc', 'Ultrafresh', 'Bias_Dry', 'Bias_Fresh',
       'Bias_Frozen', 'Bias_Ultrafresh', 'AbsError_Dry', 'AbsError_Fresh',
       'AbsError_Frozen', 'AbsError_Ultrafresh', 'SqError_Dry',
       'SqError_Fresh', 'SqError_Frozen', 'SqError_Ultrafresh',
       'RelAbsError_Dry', 'Outside_range_Dry', 'RelAbsError_Fresh',
       'Outside_range_Fresh', 'RelAbsError_Frozen',
       'Outside_range_Frozen', 'RelAbsError_Ultrafresh',
       'Outside_range_Ultrafresh'], dtype=object)

### Plots

#### Inbound dynamics

In [0]:
plot_inbound_fc(inbound_df = inbound_2022_df, actuals_col= 'Dry Actuals', fc_col = 'Dry Fc', show_out_of_range=True, outside_range_col = 'Outside_range_Dry', date_col='Dates')

In [0]:
plot_inbound_fc(inbound_df = inbound_2022_df, actuals_col= 'Dry Actuals', fc_col = 'Dry Fc', show_out_of_range=False, date_col='Dates')

In [0]:
plot_inbound_fc(inbound_df = inbound_2022_df, actuals_col= 'Frozen', fc_col = 'Frozen Fc', show_out_of_range=True, outside_range_col = 'Outside_range_Frozen', date_col='Dates')

#### Inbound errors

In [0]:
plot_inbound_error(inbound_df=inbound_2022_df, error_col='Bias_Dry',  show_out_of_range=True,  date_col='Dates', outside_range_col = 'Outside_range_Dry')

In [0]:
plot_inbound_error(inbound_df=inbound_2022_df, error_col='AbsError_Dry',  show_out_of_range=True,  date_col='Dates', outside_range_col = 'Outside_range_Dry')

In [0]:
plot_inbound_error(inbound_df=inbound_2022_df, error_col='SqError_Dry',  show_out_of_range=True,  date_col='Dates',outside_range_col = 'Outside_range_Dry')

### Global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=inbound_2022_df, cols_to_keep=['Dry Fc',	'Dry Actuals', 'Bias_Dry', 'AbsError_Dry', 'SqError_Dry', 'RelAbsError_Dry', 'Outside_range_Dry'])

<command-2032546514366438>:25: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



In [0]:
global_metrics_dry

,Metrics,Dry Fc,Dry Actuals,Bias_Dry,AbsError_Dry,SqError_Dry,RelAbsError_Dry,Outside_range_Dry
0,count,298.000000,298.000000,298.000000,298.000000,2.980000e+02,297.000000,298
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,False
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,173
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,66360.439337,64639.817651,1720.621686,6983.660229,9.947805e+07,0.131611,NaN
7,std,19931.373039,23089.057791,9840.857953,7132.830318,2.867034e+08,0.206143,NaN
8,min,0.000000,0.000000,-59204.840000,0.000000,0.000000e+00,0.000000,NaN
9,25%,55140.423206,49740.750000,-3505.689930,2321.913435,5.392652e+06,0.035599,NaN


In [0]:
# plot the frozen 
plot_inbound_fc(inbound_df = inbound_2022_df, actuals_col= 'Frozen', fc_col = 'Frozen Fc', show_out_of_range=True, outside_range_col = 'Outside_range_Frozen', date_col='Dates')


In [0]:
# inbound_2022_df

## Get days and week number function 

Assign each dates in the Dataframe it's corresponding weekdays and weeknumber 

In [0]:
def create_weekdays_df(inbound_df:pd.DataFrame,col_dates:str)->pd.DataFrame:
   """
   get_weekdays from the inbound dataframe on dates column
   Arguments
   dataframe, dates_col
   Returns 
   inbound_df dataframe with weekday as new column
   """
   inbound_df['weekday'] = inbound_df[col_dates].dt.day_name()
   return inbound_df


In [0]:
def get_week_number_df(inbound_df:pd.DataFrame, col_dates:str)->pd.DataFrame:
  """
  calculate week number, avoid duplicate by using isocalendar().week
  Args
  dataframe, date, week 
  Returns
  dataframe with a new column of week numbers
  """
  inbound_df['week_num'] = inbound_df[col_dates].dt.isocalendar().week 
  return inbound_df

In [0]:
# call the functions 
weekdays_df=create_weekdays_df(inbound_df=inbound_2022_df, col_dates='Dates')
week_num_df= get_week_number_df(inbound_df=inbound_2022_df, col_dates='Dates')


## 2. Groupby functions
Group on week_num, and weekdays 

In [0]:
def groupby_weekday_weeknum_df(inbound_df:pd.DataFrame, weekdays_col:str, outside_range_colname:str,agg_func:str)->pd.DataFrame:
  """"
  Groupby weekdays and aggregate on each outside_range + prod suffix column
  Arguments
  pd.DataFrame
  weekday column
  outside-range column and the aggregation function 
  Return 
  dataframe
  """
  wkday_wnum_df = inbound_df.groupby(weekdays_col).agg({outside_range_colname:agg_func}).reset_index()
  return wkday_wnum_df

In [0]:
wkday_wnum_df = groupby_weekday_weeknum_df(inbound_df=inbound_2022_df, weekdays_col='weekday', outside_range_colname ='Outside_range_Dry', agg_func='sum')
wkday_wnum_df

,weekday,Outside_range_Dry
0,Friday,26
1,Monday,13
2,Saturday,22
3,Sunday,0
4,Thursday,24
5,Tuesday,20
6,Wednesday,20


In [0]:

def groupby_weekday_weeknum_df(inbound_df: pd.DataFrame, weekdays_col: str, outside_range_colname: list, agg_func: str) -> pd.DataFrame:
    """
    Groupby and aggregate on each outside_range + prod suffix column
    Arguments:
    inbound_df --> pd.DataFrame
    weekdays_col --> weekday column
    outside_range_colname -- list of outside-range columns
    agg_func -- aggregation function
    Return:
    DataFrame
    """
    result_df = pd.DataFrame()
  
    # for colname in outside_range_colname:
    temp_df = inbound_df.groupby(weekdays_col).agg({outside_range_colname:agg_func for col in outside_range_colname}).reset_index()
    result_df = pd.concat([result_df, temp_df], axis=1)
    return result_df


###3. Define useful variables 

In [0]:
outside_range_colname = ['Outside_range_Dry', 'Outside_range_Fresh', 'Outside_range_Frozen', 'Outside_range_Ultrafresh']
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

In [0]:
all_prod_Category_df= groupby_weekday_weeknum_df(inbound_df=inbound_2022_df, weekdays_col='weekday', outside_range_colname='Outside_range_Dry', agg_func='sum')
all_prod_Category_df

,weekday,Outside_range_Dry
0,Friday,26
1,Monday,13
2,Saturday,22
3,Sunday,0
4,Thursday,24
5,Tuesday,20
6,Wednesday,20


In [0]:
def groupby_weeknum_df(inbound_df:pd.DataFrame,weeknum_col:str,outside_range_col,agg_func:str)->pd.DataFrame:
  """"
  Groupby weekdays and aggregate on each outside_range + prod suffix column
  Arguments
  pd.DataFrame
  weekday column
  outside-range column and the aggregation function 
  Return 
  dataframe
  """
  wk_num_df = inbound_df.groupby(weeknum_col).agg({outside_range_col:agg_func}).reset_index()
  return wk_num_df

In [0]:
wk_num_df=groupby_weeknum_df(inbound_df=inbound_2022_df, weeknum_col='week_num',outside_range_col='Outside_range_Dry',agg_func='sum')
wk_num_df.head()

,week_num,Outside_range_Dry
0,1,4
1,2,3
2,3,1
3,4,2
4,5,2


In [0]:

def groupby_weekday_week_generic(inbound_df:pd.DataFrame, weekday_col:str,Outside_range_colname:str, agg_func:str)->pd.DataFrame:
  """"
  groupby and aggregate on each outside_range + prod suffix
  return 
  dataframe
  """
  grouped_df_generic = inbound_df.groupby(weekday_col).agg({Outside_range_colname:agg_func}).reset_index()
  return grouped_df_generic


In [0]:
grouped_df_generic_mean = groupby_weekday_week_generic(inbound_df=inbound_2022_df,weekday_col='weekday',Outside_range_colname='Outside_range_Dry', agg_func='mean')
grouped_df_generic_mean

,weekday,Outside_range_Dry
0,Friday,0.509804
1,Monday,0.265306
2,Saturday,0.488889
3,Sunday,0.000000
4,Thursday,0.489796
5,Tuesday,0.392157
6,Wednesday,0.384615


In [0]:
grouped_df_generic_sum = groupby_weekday_week_generic(inbound_df=inbound_2022_df,weekday_col='weekday',Outside_range_colname='Outside_range_Fresh', agg_func='sum')
grouped_df_generic_sum

,weekday,Outside_range_Fresh
0,Friday,30
1,Monday,31
2,Saturday,29
3,Sunday,0
4,Thursday,26
5,Tuesday,33
6,Wednesday,25


In [0]:
grouped_df_generic_count = groupby_weekday_week_generic(inbound_df=inbound_2022_df,weekday_col='weekday',Outside_range_colname='Outside_range_Fresh', agg_func='count')
grouped_df_generic_count

,weekday,Outside_range_Fresh
0,Friday,51
1,Monday,49
2,Saturday,45
3,Sunday,1
4,Thursday,49
5,Tuesday,51
6,Wednesday,52


### 4. sort the weekdays in calendar order 

In [0]:
group_sorted_df = wkday_wnum_df.sort_values('weekday', key=lambda x: x.map({day: i for i, day in enumerate(weekday_order)})).reset_index(drop=True)
group_sorted_df

,weekday,Outside_range_Dry
0,Monday,13
1,Tuesday,20
2,Wednesday,20
3,Thursday,24
4,Friday,26
5,Saturday,22
6,Sunday,0


In [0]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=wk_num_df['week_num'], y=wk_num_df['Outside_range_Dry'], name='Dry'))
fig.update_layout(title='Line Plot',xaxis_title='week_num', yaxis_title='days_OoR_per_week')
fig.show()



## 2nd step Get_holidays 
This will help to evaluate the impact of holidays on the forecast 

In [0]:

def get_belgium_holidays(year:int):
    """
    Get the Belgian holiday calendar for 2022.
    Argument
    year (int): The year to retrieve the holidays.
    Returns:
    The DataFrame containing the Belgian holidays for the specified year.
    """
    belgium_holidays = holidays.Belgium(years=year)

    # Convert the holiday dictionary to a DataFrame
    holidays_list = [(date, name) for date, name in belgium_holidays.items()]
    holidays_df = pd.DataFrame(holidays_list, columns=['Dates', 'is_holiday'])

    return holidays_df
year=2022
belgium_holidays_df = get_belgium_holidays(year)
#change the datatype of belgium_holiday df
belgium_holidays_df['Dates'] = pd.to_datetime(belgium_holidays_df['Dates'], errors='coerce')

In [0]:
belgium_holidays_df = get_belgium_holidays(year)
belgium_holidays_df 

,Dates,is_holiday
0,2022-01-01,Nieuwjaarsdag
1,2022-04-17,Pasen
2,2022-04-18,Paasmaandag
3,2022-05-26,O.L.H. Hemelvaart
4,2022-06-05,Pinksteren
5,2022-06-06,Pinkstermaandag
6,2022-05-01,Dag van de Arbeid
7,2022-07-21,Nationale feestdag
8,2022-08-15,O.L.V. Hemelvaart
9,2022-11-01,Allerheiligen


In [0]:
def corresponding_week_number(belgium_holidays_df:pd.DataFrame, Dates:int)->pd.DataFrame:
  """
  calculate week number, avoid duplicate by using isocalendar().week
  Args
  dataframe, date, week 
  Returns
  dataframe with a new column of week numbers
  """
  belgium_holidays_df['week_num'] = pd.to_datetime(belgium_holidays_df[Dates]).dt.isocalendar().week 
  # drop date column
  return belgium_holidays_df

In [0]:
belgium_holidays_df = corresponding_week_number(belgium_holidays_df,'Dates')

In [0]:
 belgium_holidays_df = belgium_holidays_df.drop('Dates', axis=1)

In [0]:

def change_to_boolean(belgium_holidays_df:pd.DataFrame, is_holiday) -> pd.DataFrame:
    """
    Convert the column is holiday in the DataFrame to boolean type.
    Args:
    The DataFrame containing the column to be converted.
    column_name: The name of the column to be converted.
    Returns:
    The modified DataFrame with the converted column.
    """
    belgium_holidays_df['is_holiday'] = belgium_holidays_df['is_holiday'].astype(bool)
    return belgium_holidays_df


In [0]:

belgium_holiday_bool_df=change_to_boolean(belgium_holidays_df, 'is_holiday')
belgium_holiday_bool_df.head()

,is_holiday,week_num
0,True,52
1,True,15
2,True,16
3,True,21
4,True,22


## Merge function
The merging function joins the 2 DataFrame and matches them on Week_num

In [0]:
inbound_2022_merge= pd.merge(inbound_2022_df, belgium_holiday_bool_df, on=['week_num'], how='left')
inbound_2022_merge['is_holiday'].fillna(False, inplace=True)
# display the result
inbound_2022_merge.head()

,Dates,Dry Fc,Dry Actuals,Fresh Fc,Fresh,Frozen Fc,Frozen,Ultrafresh Fc,Ultrafresh,Bias_Dry,Bias_Fresh,Bias_Frozen,Bias_Ultrafresh,AbsError_Dry,AbsError_Fresh,AbsError_Frozen,AbsError_Ultrafresh,SqError_Dry,SqError_Fresh,SqError_Frozen,SqError_Ultrafresh,RelAbsError_Dry,Outside_range_Dry,RelAbsError_Fresh,Outside_range_Fresh,RelAbsError_Frozen,Outside_range_Frozen,RelAbsError_Ultrafresh,Outside_range_Ultrafresh,weekday,week_num,is_holiday
0,2022-01-03,85794.442754,84224.0,60969.286307,41000.0,49273.053738,5581.0,6538.217201,6538.217201,1570.442754,19969.286307,43692.053738,0.000000,1570.442754,19969.286307,43692.053738,0.000000,2.466290e+06,3.987724e+08,1.908996e+09,0.000000e+00,0.018646,False,0.487056,True,7.828714,True,0.000000,False,Monday,1,False
1,2022-01-04,65323.939111,49564.0,68517.000793,64233.0,6447.045743,6110.0,7288.014352,9673.000000,15759.939111,4284.000793,337.045743,-2384.985648,15759.939111,4284.000793,337.045743,2384.985648,2.483757e+08,1.835266e+07,1.135998e+05,5.688157e+06,0.317971,True,0.066695,False,0.055163,False,0.246561,True,Tuesday,1,False
2,2022-01-05,59060.496963,45890.0,59891.490569,61162.0,14752.884950,6061.0,8645.127518,9394.000000,13170.496963,-1270.509431,8691.884950,-748.872482,13170.496963,1270.509431,8691.884950,748.872482,1.734620e+08,1.614194e+06,7.554886e+07,5.608100e+05,0.287001,True,0.020773,False,1.434068,True,0.079718,False,Wednesday,1,False
3,2022-01-06,54098.434315,26976.0,54044.997867,55700.0,11884.161805,5042.0,8733.406013,7299.000000,27122.434315,-1655.002133,6842.161805,1434.406013,27122.434315,1655.002133,6842.161805,1434.406013,7.356264e+08,2.739032e+06,4.681518e+07,2.057521e+06,1.005428,True,0.029713,False,1.357033,True,0.196521,True,Thursday,1,False
4,2022-01-07,48633.525710,37044.0,48691.064872,49534.0,7322.811052,5979.0,7745.598367,8240.000000,11589.525710,-842.935128,1343.811052,-494.401633,11589.525710,842.935128,1343.811052,494.401633,1.343171e+08,7.105396e+05,1.805828e+06,2.444330e+05,0.312858,True,0.017017,False,0.224755,True,0.060000,False,Friday,1,False


In [0]:

def groupby_weekday_week_generic_multiple_columns(inbound_df:pd.DataFrame, weekday_col:str,Outside_range_columns:list, agg_func:str)->pd.DataFrame:
  """"
  groupby and aggregate on each outside_range + prod suffix
  return 
  dataframe
  """
  aggregation_dict = {Outside_range_colname:agg_func for Outside_range_colname in Outside_range_columns}
  grouped_df_generic = inbound_df.groupby(weekday_col).agg(aggregation_dict).reset_index()
  return grouped_df_generic

In [0]:
groupby_weekday_week_generic_multiple_columns(inbound_df=inbound_2022_df, weekday_col='weekday',Outside_range_columns=['Outside_range_Dry','Outside_range_Fresh','Outside_range_Frozen','Outside_range_Ultrafresh'], agg_func='sum')

,weekday,Outside_range_Dry,Outside_range_Fresh,Outside_range_Frozen,Outside_range_Ultrafresh
0,Friday,26,30,35,35
1,Monday,13,31,45,38
2,Saturday,22,29,32,30
3,Sunday,0,0,0,0
4,Thursday,24,26,44,41
5,Tuesday,20,33,34,40
6,Wednesday,20,25,46,26
